# Exact Diagonalization of 2D J-Q Hamiltonian
    Himadri Halder, S. Pujari Group
    2024, IIT Bombay

In [1]:
# Load the source files
include("../source/J-Q_ED.jl")

VBS_0p_structure (generic function with 1 method)

In [3]:
Nx,Ny = 4,4
mz0 = 0.0
kx,ky = 0,0
J,Q = 0.1,0.9

eng_mz0_path = joinpath("D:\\Project_Julia\\Data\\2D_JQ_$(Nx)x$(Ny)\\Energy_Eigenvec",
    "Mz=$(mz0)\\mx=$(kx)_my=$(ky)\\Energy_J=$(round(J,digits=8))_Q=$(round(Q,digits=8)).jld2")
@load eng_mz0_path energy_array
E0 = energy_array[1].energy
E1 = energy_array[2].energy

evec_array_mz0_path = joinpath("D:\\Project_Julia\\Data\\2D_JQ_$(Nx)x$(Ny)\\Energy_Eigenvec",
    "Mz=$(mz0)\\mx=$(kx)_my=$(ky)\\Eigenvec_J=$(round(J,digits=8))_Q=$(round(Q,digits=8)).jld2")
evec_array_mz0 = @load evec_array_mz0_path evecs

1-element Vector{Symbol}:
 :evecs

In [4]:
energy_array

10-element Vector{@NamedTuple{energy::Float64, kx::Int64, ky::Int64, z::Int64}}:
 (energy = -14.520426076452235, kx = 0, ky = 0, z = 1)
 (energy = -12.2215699469465, kx = 0, ky = 0, z = 1)
 (energy = -11.012572126316547, kx = 0, ky = 0, z = 1)
 (energy = -10.115760576070574, kx = 0, ky = 0, z = 1)
 (energy = -9.967665140980268, kx = 0, ky = 0, z = -1)
 (energy = -9.967665140980232, kx = 0, ky = 0, z = -1)
 (energy = -9.486203393038037, kx = 0, ky = 0, z = 1)
 (energy = -9.385134690723259, kx = 0, ky = 0, z = 1)
 (energy = -8.749609553042937, kx = 0, ky = 0, z = 1)
 (energy = -8.6963643971467, kx = 0, ky = 0, z = 1)

In [5]:
Nx,Ny = 4,4
mz = 0.0
kx,ky = 2,2
z = -1

basis_file = joinpath("D:\\Project_Julia\\Data\\2D_JQ_$(Nx)x$(Ny)\\Basis\\Mz_kx_ky_z_basis",
                    "Mz=$(mz)\\Basis_mx=$(kx)_my=$(ky)_z=$(z).jld2")
if isfile(basis_file)
    basis_info = JLD2.load(basis_file)
    basis_list = basis_info["basis_list"]
    Na_list = basis_info["Na_list"]
end;

In [8]:
basis_list

386-element Vector{Int64}:
   383
   447
   479
   495
   503
   831
   863
   879
   887
   891
   893
   894
   927
     ⋮
 13718
 13724
 13733
 13734
 13737
 13738
 13740
 13765
 13766
 13769
 21850
 23130

In [54]:
Nx,Ny = 6,4
mz = 0.0
kx,ky = 0,0
z = 1

x_lattice = lattice_x(Nx,Ny)
y_lattice = lattice_y(Nx,Ny)

@time H_size,basis_list,D_list,Na_list = gen_basis_mz_k_z(Nx,Ny,mz,kx,ky,z,x_lattice,y_lattice);

  4.345320 seconds (33.34 M allocations: 2.257 GiB, 4.96% gc time)


In [11]:
Nx,Ny = 4,4
mz = 0.0
kx,ky = 0,0
z = 1
J,Q = 0.2,0.8
use_sparse = true

x_lattice = lattice_x(Nx,Ny)
y_lattice = lattice_y(Nx,Ny)
Jbonds,Qbonds = bond_list(Nx,Ny)

basis_file = joinpath("D:\\Project_Julia\\Data\\2D_JQ_$(Nx)x$(Ny)\\Basis\\Mz_kx_ky_z_basis",
        "Mz=$(mz)\\Basis_mx=$(kx)_my=$(ky)_z=$(z).jld2")
if isfile(basis_file)
    basis_info = JLD2.load(basis_file)
    basis_list = basis_info["basis_list"]
    Na_list = basis_info["Na_list"]

    #Pre-allocate memory for in-place modification later
    basis_copy = x_tstate = xy_tstate = flipped_basis = similar(state_bin(basis_list[1],Nx*Ny))
    
    @time Ham_J,H_size,basis_list,Na_list = 
    JQ_Ham_J(J,Nx,Ny,kx,ky,z,basis_list,Na_list,x_lattice,y_lattice,Jbonds,use_sparse,basis_copy,
             x_tstate,xy_tstate)
    
    @time Ham_Q,H_size,basis_list,Na_list = 
    JQ_Ham_Q(Q,Nx,Ny,kx,ky,z,basis_list,Na_list,x_lattice,y_lattice,Qbonds,use_sparse,basis_copy,
        flipped_basis,x_tstate,xy_tstate)
end;

  0.096103 seconds (342.38 k allocations: 16.599 MiB, 70.45% compilation time)
  0.079934 seconds (567.75 k allocations: 28.114 MiB, 8.21% gc time, 10.71% compilation time: 100% of which was recompilation)


In [5]:
Nx,Ny = 4,4
kx,ky = 0,0
mz = 0.0
z1 = 1
J,Q = 0.1,0.9
use_sparse = true

J_path1 = joinpath("D:\\Project_Julia\\Data\\2D_JQ_$(Nx)x$(Ny)\\Ham_matrices",
                   "Mz=$(mz)\\mx=$(kx)_my=$(ky)_z=$(z1)\\" ,
                   (use_sparse ? "Sparse" : "Dense") * "_ham_J=$(round(J,digits=8)).jld2")
Q_path1 = joinpath("D:\\Project_Julia\\Data\\2D_JQ_$(Nx)x$(Ny)\\Ham_matrices",
                   "Mz=$(mz)\\mx=$(kx)_my=$(ky)_z=$(z1)\\" ,
                   (use_sparse ? "Sparse" : "Dense") * "_ham_Q=$(round(Q,digits=8)).jld2")
Ham_J = JLD2.load(J_path1)["Ham_J"]
Ham_Q = JLD2.load(Q_path1)["Ham_Q"]
Ham = Ham_J+Ham_Q

441×441 SparseMatrixCSC{ComplexF64, Int64} with 9645 stored entries:
⎡⣿⣿⣷⣷⣮⣦⡐⣶⢶⣏⡝⡗⠢⢤⣘⣃⠏⠀⠀⠄⢠⣰⡣⣤⠄⢁⠥⣝⠦⣀⢀⠀⢨⢦⡒⡴⣤⢠⢠⡄⎤
⎢⢽⣿⣿⣿⡿⣋⠛⠿⣿⢏⡘⣙⠀⠀⠀⣙⣛⢄⣀⣣⡙⡘⢋⠛⠀⢀⠃⠋⠂⢉⠠⣌⢈⡛⡃⠀⣥⢤⠀⠁⎥
⎢⠪⣿⡿⢫⢻⣶⣾⣾⣟⡄⡉⠉⣀⣂⣐⣼⠍⣤⢤⡥⡽⣕⢎⢬⡶⣶⢧⢴⣳⢦⠹⡌⢚⢏⠆⠀⠀⠀⠁⠀⎥
⎢⢰⣬⣿⡄⣺⣿⣵⣿⣾⡟⠉⣄⠥⠍⠡⣳⣤⣁⡁⡉⣈⣘⣻⣲⡀⠐⣿⣐⡂⢲⡾⢷⢲⡺⡖⠀⠀⠀⠀⠀⎥
⎢⡼⢷⡿⢟⠛⠽⣾⠿⠿⢇⣀⣷⣐⠸⠸⡲⣓⡈⢡⢬⠭⠉⡍⠽⣆⣂⣑⡀⠒⠚⠙⠚⠈⠛⠃⠀⠀⠀⠀⠀⎥
⎢⢷⠭⣖⢨⡇⠈⠃⢤⢤⣼⣿⣿⠨⠨⢵⡂⣽⢈⠹⠢⠨⡁⣕⡐⣻⢷⣨⣟⠨⠬⡾⠐⡯⡖⢂⠀⠀⠀⠀⠀⎥
⎢⠈⣆⠀⠀⠠⢸⡅⠇⣐⡘⡂⡂⣿⣿⣿⣿⣬⡻⣼⢕⢦⣻⡚⣲⣿⡧⡞⢖⠹⣬⢳⡹⡠⠀⠈⣛⣚⠒⣒⠀⎥
⎢⠶⢸⣄⢠⣐⣼⢥⣢⢲⡢⠱⠳⣿⣿⣿⣿⡝⡵⡿⣿⣖⢿⣾⣔⠽⠷⣮⣻⡘⣫⣦⢿⠖⣄⡆⠁⠞⠩⠅⠆⎥
⎢⠋⠁⠛⢜⠃⣥⠄⢻⡙⠸⡓⢛⣦⡻⢗⡭⣻⣾⣽⣿⣽⣭⢁⢋⣷⣿⣿⢷⢬⣁⠾⣔⢳⣟⡅⠒⡰⣤⡄⠠⎥
⎢⠀⠄⠤⣸⠄⡷⡅⠨⡁⣖⠳⡂⢖⢟⣿⣯⣷⣿⣿⣿⣿⣭⣿⣬⣛⣾⣿⠻⣿⡜⣦⣿⣺⣞⡄⠊⡘⣓⣆⠠⎥
⎢⢀⣲⣓⠨⢗⢯⣂⢸⡇⠃⠆⠢⣬⣳⣼⣝⡗⣿⡟⣿⢿⣷⣽⣾⡷⠤⣶⢹⣋⣻⣽⣝⢸⣷⣇⣶⡰⠴⠶⠂⎥
⎢⠉⣮⣯⠐⡊⣕⢻⣺⣇⡍⢑⠹⢺⣨⢚⢿⡥⢐⡛⣿⣳⣿⣿⣿⡏⣽⣻⢤⣦⣔⣿⣿⡿⣷⡊⠉⡘⠺⣻⠅⎥
⎢⠄⢁⠀⢀⢸⣯⢀⠈⠨⢹⢿⣞⠿⡿⢷⡇⣽⣿⣻⣼⠙⡏⣏⣭⣿⣿⣷⣽⢷⢿⢑⡘⣟⣥⣟⡤⢤⡀⠀⠀⎥
⎢⣅⢧⡭⠀⢉⣗⢛⢻⠑⠸⣦⢾⢺⢍⣮⣻⢿⣟⣿⡛⣜⣛⠛⣞⣝⣿⡛⣬⣯⣽⣟⠟⣬⣻⡁⡭⣨⢞⣷⡄⎥
⎢⠈⢣⡌⢀⠹⣞⢨⣈⣸⠀⡂⡆⡓⣦⡶⣨⠆⢳⣛⠿⣯⣸⢈⢿⣽⣗⣏⣿⣿⣿⣹⣧⢶⣷⠏⣫⢔⢲⠦⠀⎥
⎢⠀⠐⡀⢦⡓⠦⢾⣏⣳⠀⢚⠋⣝⡲⣬⣟⢚⢧⣬⣿⣗⢿⣿⣿⣑⠰⣿⠝⠷⣾⣿⣿⢻⣿⡎⠐⡒⢜⢧⠨⎥
⎢⠢⣖⣦⠰⡾⢔⣸⡲⣦⠀⢫⠯⠀⠊⠘⢥⣽⢶⣺⢾⢶⣶⢿⣯⠟⣽⣦⣻⢼⣷⣿⣶⣿⣿⡧⠴⡦⢴⣖⡆⎥
⎢⢘⡬⠉⠈⠈⠁⠘⠉⠉⠀⠈⠐⣦⢠⠌⠉⢡⠉⡠⠉⢩⣽⡎⠈⠛⡽⡅⡬⡯⣡⢊⠉⢉⡏⣿⣿⣿⣯⣦⠤⎥
⎢⠀⣛⠁⣟⠀⠀⠀⠀⠀⠀⠀⠀⢺⠘⡞⡁⠐⣮⢶⢨⢐⡎⣲⡈⠀⠳⣢⢞⢰⣑⣘⢌⢈⣏⡿⣿⣿⣿⣿⣲⎥
⎣⠀⠶⠄⠀⠁⠀⠀⠀⠀⠀⠀⠀⠘⠘⠡⠅⠀⡉⠈⡙⠸⠃⠟⠞⠀⠀⠙⠿⠈⠃⡉⡓⠸⠽⠈⡟⢻⣻⡿⢏⎦

In [77]:
Nx,Ny = 6,4
mz = 0.0
kx,ky = 0,0
z = 1

basis_file = joinpath("D:\\Project_Julia\\Data\\2D_JQ_$(Nx)x$(Ny)\\Basis\\Mz_kx_ky_z_basis",
                    "Mz=$(mz)\\Basis_mx=$(kx)_my=$(ky)_z=$(z).jld2")
if isfile(basis_file)
    basis_info = JLD2.load(basis_file)
    basis_list = basis_info["basis_list"]
    Na_list = basis_info["Na_list"]
    basis_size = basis_info["H_size"]
    println(basis_size)
end;

56664


In [78]:
use_sparse = true

x_lattice = lattice_x(Nx,Ny)
y_lattice = lattice_y(Nx,Ny)
Jbonds,Qbonds = bond_list(Nx,Ny)

basis_copy = x_tstate = xy_tstate = flipped_basis = similar(state_bin(basis_list[1],Nx*Ny))

@time Ham_J,H_size,basis_list,Na_list = 
JQ_Ham_J(J,Nx,Ny,kx,ky,z,basis_list,Na_list,x_lattice,y_lattice,Jbonds,use_sparse,basis_copy,
             x_tstate,xy_tstate);

 65.222762 seconds (92.34 M allocations: 4.223 GiB, 1.12% gc time)
